In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import json
import os

import analysis.pipelines.population_spikes as ps

RESULT_PATH = "simulation_results"
OUTPUT_PATH = "analysis_results"

In [2]:
ps.set_variables(RESULT_PATH=RESULT_PATH, OUTPUT_PATH=OUTPUT_PATH)

PN_pop_names = ps.PN_pop_names
ITN_pop_names = ps.ITN_pop_names

In [51]:
wave_pop = 'ITN'

waves = ['gamma', 'beta']
pop_groups = ['PN stimulated', 'PN rest'] + ITN_pop_names
stats = ['mean', 'stdev']
stat_func = {'mean': np.mean, 'stdev': np.std}

# define function for retrieving unit ids in each group
def get_grp_ids(plv_ds):
    ids = {}
    idx = plv_ds.assy_id.values >= 0
    ids['PN stimulated'] = np.concatenate([plv_ds[p].values for p in plv_ds.PN_names[idx].values])
    ids['PN rest'] = np.concatenate([plv_ds[p].values for p in plv_ds.PN_names[~idx].values])
    for p in ITN_pop_names:
        ids[p] = plv_ds[p].values
    return ids

columns = pd.MultiIndex.from_product((waves, pop_groups, stats))
plv_list = []
tr_list = []

In [3]:
with open(os.path.join(OUTPUT_PATH, 'trials_ordered.json'), 'r') as f:
    trials_ordered = json.load(f)

trials = trials_ordered

In [57]:
for tr in trials:
    file = os.path.join(ps.FR_ENTR_PATH, tr + '.nc')
    if not os.path.isfile(file):
        continue
    tr_list.append(tr)
    plv_ds = xr.load_dataset(file)
    grp_ids = get_grp_ids(plv_ds)
    plv_stat = pd.Series(0., index=columns)
    for grp, ids in grp_ids.items():
        plv = plv_ds.PLV.sel(unit_id=ids, wave_population=wave_pop)
        for w in waves:
            for stat in stats:
                plv_stat.loc[(w, grp, stat)] = stat_func[stat](plv.values)
    plv_list.append(plv_stat.values)

plv_df = pd.DataFrame(plv_list, index=pd.Index(tr_list, name='trial'), columns=columns)

In [58]:
plv_df

gamma                                          \
                  PN stimulated             PN rest                 FSI   
                           mean     stdev      mean     stdev      mean   
trial                                                                     
ramp_a0_t1.0_rand      0.055629  0.076272  0.100116  0.197756  0.054167   
ramp_a0_t1.0_rand      0.055629  0.076272  0.100116  0.197756  0.054167   

                                                         beta            \
                                  LTS           PN stimulated             
                      stdev      mean     stdev          mean     stdev   
trial                                                                     
ramp_a0_t1.0_rand  0.098634  0.333352  0.219691      0.055629  0.076272   
ramp_a0_t1.0_rand  0.098634  0.333352  0.219691      0.055629  0.076272   

                                                                               
                    PN rest                 FSI                 LTS            
                       mean     stdev      mean     stdev      mean     stdev  
trial                                                                          
ramp_a0_t1.0_rand  0.100116  0.197756  0.054167  0.098634  0.333352  0.219691  
ramp_a0_t1.0_rand  0.100116  0.197756  0.054167  0.098634  0.333352  0.219691